To visualize tensorflow model, we need to write log files that tensor board can read for visualizations.

In [1]:
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import *

training_data_df = pd.read_csv("sales_data_training_scaled.csv")

X = training_data_df.drop('total_earnings', axis=1).values
Y = training_data_df[['total_earnings']].values

print(X)
print(X.shape)

RUN_NAME ='run 2 with 100 input nodes'#Just change this string everytime you twesk the network

# Define the model
model = Sequential()
model.add(Dense(100, input_dim=9, activation='relu', name='layer_1'))#nane of each layer is augmented to make visualizations more clear
model.add(Dense(100, activation='relu', name='layer_2'))
model.add(Dense(50, activation='relu', name='layer_3'))
model.add(Dense(1, activation='linear', name='output_layer'))
model.compile(loss='mean_squared_error', optimizer='adam')

# Create a TensorBoard logger
logger = keras.callbacks.TensorBoard(
    log_dir="log/{}".format(RUN_NAME),
    write_graph= True, #by default keras will only log details of training process but it will not log details of the model
    histogram_freq=5 #information about how each layer of our neural network is working. Here 5 means that for every 5 passes keras will write statistics              
)




# Load the separate test data set
test_data_df = pd.read_csv("sales_data_testing_scaled.csv")

X_test = test_data_df.drop('total_earnings', axis=1).values
Y_test = test_data_df[['total_earnings']].values
print(X_test.shape)
# Train the model


model.fit(
    X,
    Y,
    epochs=50,
    shuffle=True,
    verbose=2,
    validation_data=(X_test,Y_test),
    callbacks=[logger]#an array of functions that we want keras to call after each pass during training phase
)

test_error_rate = model.evaluate(X_test, Y_test, verbose=0)
print("The mean squared error (MSE) for the test data set is: {}".format(test_error_rate))


Using TensorFlow backend.


[[0.5        1.         0.         ... 0.         0.         1.        ]
 [0.83333333 0.         0.         ... 1.         0.         0.5       ]
 [0.33333333 0.         0.         ... 1.         0.         0.5       ]
 ...
 [0.83333333 0.         1.         ... 0.         0.         1.        ]
 [0.5        1.         1.         ... 0.         1.         0.        ]
 [0.33333333 1.         0.         ... 0.         1.         0.5       ]]
(1000, 9)
(400, 9)
Train on 1000 samples, validate on 400 samples
Epoch 1/50
 - 1s - loss: 0.0096 - val_loss: 0.0015
Epoch 2/50
 - 0s - loss: 9.6673e-04 - val_loss: 4.3450e-04
Epoch 3/50
 - 0s - loss: 3.3325e-04 - val_loss: 2.8350e-04
Epoch 4/50
 - 0s - loss: 1.9976e-04 - val_loss: 2.1178e-04
Epoch 5/50
 - 0s - loss: 1.8631e-04 - val_loss: 2.0051e-04
Epoch 6/50
 - 0s - loss: 1.2207e-04 - val_loss: 1.1979e-04
Epoch 7/50
 - 0s - loss: 8.5315e-05 - val_loss: 1.0833e-04
Epoch 8/50
 - 0s - loss: 5.9818e-05 - val_loss: 7.7912e-05
Epoch 9/50
 - 0s - loss: 4

# Tensor Flow  Code

In [2]:
import tensorflow as tf
model_builder = tf.saved_model.builder.SavedModelBuilder("exported_model")
inputs = {
    'input': tf.saved_model.utils.build_tensor_info(model.input)
}
outputs = {
    'earnings': tf.saved_model.utils.build_tensor_info(model.output)
}

signature_def = tf.saved_model.signature_def_utils.build_signature_def(
    inputs=inputs,
    outputs=outputs,
    method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME
)

model_builder.add_meta_graph_and_variables(
    K.get_session(),
    tags=[tf.saved_model.tag_constants.SERVING],
    signature_def_map={
        tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY: signature_def
    }
)

model_builder.save()

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: exported_model/saved_model.pb


b'exported_model/saved_model.pb'

- if any error occur we may be required to restart kernel
- to start tensorboard use tensorboard --logdir=directory
- In the scalar tab click on LOSS. Y axis represents Loss, X axis #of passes
- copy paste port num to http://localhost:6006